# Healthcare Accesibility per Neighborhood in Toronto

## Introduction

Physical healthcare accesibility is defined by World Health Organization as “the availability of good health services within reasonable reach of those who need them and of opening hours, appointment systems and other aspects of service organization and delivery that allow people to obtain the services when they need them”[1]. This is an important aspect for people wellbeing and city government must guarantee access to health services to all population. 

The accurate measurement of spatial accessibility to health care is problematic chiefly because there is seldom any predetermined assignment or single pathway between individuals and specific health care services. For this project I will assume that metropolitans areas have services densely located and most individuals will access health services in close proximity. I will use the two-step floating catchment area (2SFCA) with continous distance decay function as proposed by McGrail MR [2] and try to assess the accessibility for every neighborhood in Toronto based on the distance from neihgborhoods to hospitals and emergency rooms and the population of the neighborhoods.

## Data

For this project I will be using foursquare API to obtain the location for the Hospitals and Emergency Rooms in Toronto, Canada. From https://open.toronto.ca/ I obtained the Neighbourhood Profiles dataset which cointains demographics data, from the 2016 Census, such as population, income, education, etc for 140 neighbourhoods. I will also use the geojson file from open toronto data and will use Open Street Map data to estimate the distance from the centroid of the neighbourhoods to the venues from Foursquare.

In [56]:
import folium
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import geopandas as gpd

In [57]:
#read census data as pandas Dataframe
#toronto_census = pd.read_csv('neighbourhood-profiles-2016-csv.csv')
toronto_census.head(10)

,_id,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,...,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,1,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,NaN,129,128,20,95,...,37,7,137,64,60,94,100,97,27,31
1,2,Neighbourhood Information,Neighbourhood Information,City of Toronto,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,...,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,Emerging Neighbourhood
2,3,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2016","2,731,571","29,113","23,757","12,054","30,526",...,"16,936","22,156","53,485","12,541","7,865","14,349","11,817","12,528","27,593","14,804"
3,4,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2011","2,615,060","30,279","21,988","11,904","29,177",...,"15,004","21,343","53,350","11,703","7,826","13,986","10,578","11,652","27,713","14,687"
4,5,Population,Population and dwellings,Census Profile 98-316-X2016001,Population Change 2011-2016,4.50%,-3.90%,8.00%,1.30%,4.60%,...,12.90%,3.80%,0.30%,7.20%,0.50%,2.60%,11.70%,7.50%,-0.40%,0.80%
5,6,Population,Population and dwellings,Census Profile 98-316-X2016001,Total private dwellings,"1,179,057","9,371","8,535","4,732","18,109",...,"8,054","8,721","19,098","5,620","3,604","6,185","6,103","7,475","11,051","5,847"
6,7,Population,Population and dwellings,Census Profile 98-316-X2016001,Private dwellings occupied by usual residents,"1,112,929","9,120","8,136","4,616","15,934",...,"7,549","8,509","18,436","5,454","3,449","5,887","5,676","7,012","10,170","5,344"
7,8,Population,Population and dwellings,Census Profile 98-316-X2016001,Population density per square kilometre,"4,334","3,929","3,034","2,435","10,863",...,"5,820","4,007","4,345","7,838","6,722","8,541","7,162","10,708","2,086","2,451"
8,9,Population,Population and dwellings,Census Profile 98-316-X2016001,Land area in square kilometres,630.2,7.41,7.83,4.95,2.81,...,2.91,5.53,12.31,1.6,1.17,1.68,1.65,1.17,13.23,6.04
9,10,Population,Age characteristics,Census Profile 98-316-X2016001,Children (0-14 years),"398,135","3,840","3,075","1,760","2,360",...,"1,785","3,555","9,625","2,325","1,165","1,860","1,800","1,210","4,045","1,960"


In [58]:
#read geojson as pandas Dataframe
toronto_hoods = gpd.read_file('Neighbourhoods.geojson')
toronto_hoods.head()

,_id,AREA_ID,AREA_ATTR_ID,PARENT_AREA_ID,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_NAME,AREA_DESC,X,Y,LONGITUDE,LATITUDE,OBJECTID,Shape__Area,Shape__Length,geometry
0,7001,25886861,25926662,49885,94,94,Wychwood (94),Wychwood (94),None,None,-79.425515,43.676919,16491505,3.217960e+06,7515.779658,"POLYGON ((-79.43592 43.68015, -79.43492 43.680..."
1,7002,25886820,25926663,49885,100,100,Yonge-Eglinton (100),Yonge-Eglinton (100),None,None,-79.403590,43.704689,16491521,3.160334e+06,7872.021074,"POLYGON ((-79.41096 43.70408, -79.40962 43.704..."
2,7003,25886834,25926664,49885,97,97,Yonge-St.Clair (97),Yonge-St.Clair (97),None,None,-79.397871,43.687859,16491537,2.222464e+06,8130.411276,"POLYGON ((-79.39119 43.68108, -79.39141 43.680..."
3,7004,25886593,25926665,49885,27,27,York University Heights (27),York University Heights (27),None,None,-79.488883,43.765736,16491553,2.541821e+07,25632.335242,"POLYGON ((-79.50529 43.75987, -79.50488 43.759..."
4,7005,25886688,25926666,49885,31,31,Yorkdale-Glen Park (31),Yorkdale-Glen Park (31),None,None,-79.457108,43.714672,16491569,1.156669e+07,13953.408098,"POLYGON ((-79.43969 43.70561, -79.44011 43.705..."


In [59]:
#neighborhood _id in toronto_hoods is preceded by a 7 so added 7000 to _id in the density dataframe to be able to match 
hood_density = pd.DataFrame()
hood_density['_id'] = toronto_census.iloc[0, 6:].astype(int) + 7000
hood_density['pop_density'] = toronto_census.iloc[7, 6:].str.replace(',','').astype(int)

hood_density.head()

,_id,pop_density
Agincourt North,7129,3929
Agincourt South-Malvern West,7128,3034
Alderwood,7020,2435
Annex,7095,10863
Banbury-Don Mills,7042,2775


In [60]:
toronto_coord = [43.6532, -79.3832]
map_toronto = folium.Map(location=toronto_coord, zoom_start=10.8)
map_toronto.choropleth(geo_data='Neighbourhoods.geojson',
                 data=hood_density,
                 columns=['_id', 'pop_density'],
                 name='choropleth',
                 key_on='properties._id',
                 fill_color='YlGn',
                 fill_opacity=0.7,
                 line_opacity=0.2,
                 legend_name='Population Density per Square Meter')

folium.LayerControl().add_to(map_toronto)

url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
state_geo = f'{url}/us-states.json'
print(state_geo)

https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json


In [61]:
map_toronto

##### Choropleth map showing population density from Toronto census data

## Notes:

[1] Universal health coverage and universal access, Bulletin of the World Health Organization 2013; 91:546–546A

[2] McGrail MR. Spatial accessibility of primary health care utilising the two step floating catchment area method: an assessment of recent improvements. Int J Health Geogr. 2015;11:50.